In [1]:
# ===============================
# Portfolio Data Cleaning Project
# Maternal, Child & Adolescent Health Dataset
# ===============================

# --- Step 1: Import libraries ---
import pandas as pd
import numpy as np
from google.colab import files
import io

# --- Step 2: Upload file from your PC ---
print("📂 Please upload your dataset CSV file")
uploaded = files.upload()

# Automatically detect the uploaded file name
filename = list(uploaded.keys())[0]
print(f"\n✅ File uploaded successfully: {filename}")

# Load the dataset
df = pd.read_csv(io.BytesIO(uploaded[filename]), low_memory=False)
print(f"Initial shape: {df.shape}")

# --- Step 3: Clean column names ---
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
    .str.replace("/", "_")
)

# --- Step 4: Inspect data ---
print("\n=== Data Info ===")
df.info()
print("\n=== Missing Values Summary ===")
print(df.isna().sum().sort_values(ascending=False).head(10))

# --- Step 5: Drop columns with too many missing values (>70%) ---
missing_ratio = df.isna().mean()
drop_cols = missing_ratio[missing_ratio > 0.7].index
print(f"\nDropping columns with >70% missing values: {list(drop_cols)}")
df.drop(columns=drop_cols, inplace=True)

# --- Step 6: Convert numeric columns safely ---
num_cols = [
    "year", "denominator", "number_with_outcome",
    "sort_1000_or_more", "unique_row_id", "latest_data"
]
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# --- Step 7: Handle missing values ---
# Fill categorical columns with "Unknown"
cat_cols = df.select_dtypes(include=["object"]).columns
df[cat_cols] = df[cat_cols].fillna("Unknown")

# Fill numeric columns with median
num_cols = df.select_dtypes(include=[np.number]).columns
for col in num_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

# --- Step 8: Remove duplicates ---
if "unique_row_id" in df.columns:
    before = len(df)
    df = df.drop_duplicates(subset=["unique_row_id"], keep="first")
    print(f"\nRemoved {before - len(df)} duplicate rows based on unique_row_id")

# --- Step 9: Feature Engineering ---
if {"number_with_outcome", "denominator"}.issubset(df.columns):
    df["outcome_rate"] = df["number_with_outcome"] / df["denominator"].replace({0: np.nan})
    df["outcome_rate_pct"] = (df["outcome_rate"] * 100).round(2)

if "year" in df.columns:
    df["year_bucket"] = df["year"].apply(lambda y: str(int(y)) if not pd.isna(y) else "Unknown")

if "topic" in df.columns:
    df["topic_short"] = df["topic"].astype(str).str.slice(0, 80)

# --- Step 10: Detect Outliers (Z-score) ---
if "denominator" in df.columns:
    mean_val = np.nanmean(df["denominator"])
    std_val = np.nanstd(df["denominator"])
    z = (df["denominator"] - mean_val) / std_val
    df["denominator_outlier"] = z.abs() > 4
    print(f"Marked {df['denominator_outlier'].sum()} outliers in 'denominator'.")

# --- Step 11: Example Pivot Table ---
if {"topic_short", "year_bucket", "outcome_rate"}.issubset(df.columns):
    pivot = pd.pivot_table(df, index="topic_short", columns="year_bucket", values="outcome_rate", aggfunc="mean")
    print("\n=== Example Pivot Table (Top 5 Topics) ===")
    print(pivot.head(5))

# --- Step 12: Example NumPy Operations ---
if "number_with_outcome" in df.columns:
    arr = df["number_with_outcome"].to_numpy(dtype=float)
    percentiles = np.percentile(arr[~np.isnan(arr)], [0, 25, 50, 75, 100])
    print("\n=== NumPy Percentiles for 'number_with_outcome' ===")
    print(dict(zip(["0%", "25%", "50%", "75%", "100%"], percentiles)))

# --- Step 13: Final summary ---
print("\n✅ Cleaning complete!")
print(f"Final shape: {df.shape}")
print("\n=== Sample of Cleaned Data ===")
print(df.head(5))

# --- Step 14: Save and download cleaned dataset ---
cleaned_filename = "cleaned_maternal_health.csv"
df.to_csv(cleaned_filename, index=False)
print(f"\n📁 Cleaned file saved as: {cleaned_filename}")

files.download(cleaned_filename)


📂 Please upload your dataset CSV file


Saving Maternal__Child__and_Adolescent_Health_Needs_Assessment__2023-2024.csv to Maternal__Child__and_Adolescent_Health_Needs_Assessment__2023-2024.csv

✅ File uploaded successfully: Maternal__Child__and_Adolescent_Health_Needs_Assessment__2023-2024.csv
Initial shape: (62714, 28)

=== Data Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62714 entries, 0 to 62713
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age_group                     62713 non-null  object 
 1   sex                           62714 non-null  object 
 2   topic                         62713 non-null  object 
 3   data_source                   62714 non-null  object 
 4   health_condition_data_source  62713 non-null  object 
 5   period                        62713 non-null  object 
 6   year                          32079 non-null  float64
 7   denominator                   62526 non-null  float64
 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>